In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

(x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)   #1을 추가해서 28*28*1 해서 reshape많이해줌
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)   #원핫인코딩
y_test=tf.keras.utils.to_categorical(y_test,10)

#작은필터를 여러개 쓰는게 더 좋다
#필터의 개수를 계속 늘려가라
#->vggnet 핵심 개념

cnn=Sequential()
cnn.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
cnn.add(Conv2D(64, (3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation='softmax'))



In [ ]:
cnn.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
hist=cnn.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test), verbose=2)
cnn.save('saved_cnn.h5')


Epoch 1/10
469/469 - 249s - loss: 0.5234 - accuracy: 0.8167 - val_loss: 0.3389 - val_accuracy: 0.8760 - 249s/epoch - 531ms/step
Epoch 2/10
469/469 - 205s - loss: 0.3440 - accuracy: 0.8761 - val_loss: 0.2864 - val_accuracy: 0.8948 - 205s/epoch - 436ms/step
Epoch 3/10
469/469 - 192s - loss: 0.2993 - accuracy: 0.8921 - val_loss: 0.2611 - val_accuracy: 0.9030 - 192s/epoch - 409ms/step
Epoch 4/10
469/469 - 232s - loss: 0.2694 - accuracy: 0.9020 - val_loss: 0.2577 - val_accuracy: 0.9056 - 232s/epoch - 495ms/step
Epoch 5/10
469/469 - 203s - loss: 0.2444 - accuracy: 0.9103 - val_loss: 0.2468 - val_accuracy: 0.9112 - 203s/epoch - 433ms/step
Epoch 6/10
469/469 - 183s - loss: 0.2265 - accuracy: 0.9163 - val_loss: 0.2321 - val_accuracy: 0.9150 - 183s/epoch - 391ms/step
Epoch 7/10
469/469 - 223s - loss: 0.2115 - accuracy: 0.9226 - val_loss: 0.2315 - val_accuracy: 0.9180 - 223s/epoch - 476ms/step
Epoch 8/10
469/469 - 212s - loss: 0.1977 - accuracy: 0.9266 - val_loss: 0.2192 - val_accuracy: 0.9212 - 

In [ ]:
#저장된 모델 불러오기

res_cnn=cnn.evaluate(x_test, y_test, verbose=0)
print("1, cnn moel 정확률은", res_cnn[1]*100)
print(' res_cnn =', res_cnn, '\n')

saved_cnn= tf.keras.models.load_model('saved_cnn.h5')
res_saved_cnn=saved_cnn.evaluate(x_test, y_test, verbose=0)
print("2. saved model 정확률은", res_saved_cnn[1]*100)
print(' res_saved_cnn =', res_saved_cnn)

1, cnn moel 정확률은 92.30999946594238
 res_cnn = [0.21299594640731812, 0.9230999946594238] 

2. saved model 정확률은 92.30999946594238
 res_saved_cnn = [0.21299594640731812, 0.9230999946594238]


In [ ]:
#Callback함수를 이용한 최고 성능의 모델 저장(val_loss)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'best_cnn.h5', save_best_only=True)

hist=cnn.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test),
             callbacks=[checkpoint_callback], verbose=2)

#성능이 좋은 n번째 epoch까지 저장함..


Epoch 1/10
469/469 - 172s - loss: 0.1600 - accuracy: 0.9383 - val_loss: 0.2236 - val_accuracy: 0.9227 - 172s/epoch - 367ms/step
Epoch 2/10
469/469 - 175s - loss: 0.1495 - accuracy: 0.9437 - val_loss: 0.2166 - val_accuracy: 0.9248 - 175s/epoch - 374ms/step
Epoch 3/10
469/469 - 177s - loss: 0.1436 - accuracy: 0.9461 - val_loss: 0.2186 - val_accuracy: 0.9261 - 177s/epoch - 376ms/step
Epoch 4/10
469/469 - 171s - loss: 0.1336 - accuracy: 0.9498 - val_loss: 0.2266 - val_accuracy: 0.9239 - 171s/epoch - 365ms/step
Epoch 5/10
469/469 - 173s - loss: 0.1286 - accuracy: 0.9504 - val_loss: 0.2404 - val_accuracy: 0.9213 - 173s/epoch - 368ms/step
Epoch 6/10
469/469 - 177s - loss: 0.1246 - accuracy: 0.9517 - val_loss: 0.2234 - val_accuracy: 0.9271 - 177s/epoch - 378ms/step
Epoch 7/10
469/469 - 177s - loss: 0.1164 - accuracy: 0.9554 - val_loss: 0.2348 - val_accuracy: 0.9284 - 177s/epoch - 376ms/step
Epoch 8/10
469/469 - 171s - loss: 0.1111 - accuracy: 0.9575 - val_loss: 0.2284 - val_accuracy: 0.9251 - 

In [ ]:
best_cnn= tf.keras.models.load_model('best_cnn.h5')
res_best_cnn=best_cnn.evaluate(x_test, y_test,verbose=0)
print("3. best model 정확률은", res_best_cnn[1]*100)
print('  res_best_cnn =', res_best_cnn)

3. best model 정확률은 92.47999787330627
  res_best_cnn = [0.21658356487751007, 0.9247999787330627]


In [ ]:
#Callback함수를 이용한 최고 성능의 모델 저장(val_accuracy)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'best_acc_cnn.h5', monitor='val_accuracy', save_best_only=True)

hist=cnn.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test),
             callbacks=[checkpoint_callback], verbose=2)


Epoch 1/10
469/469 - 176s - loss: 0.0966 - accuracy: 0.9624 - val_loss: 0.2421 - val_accuracy: 0.9306 - 176s/epoch - 375ms/step
Epoch 2/10
469/469 - 170s - loss: 0.0932 - accuracy: 0.9641 - val_loss: 0.2517 - val_accuracy: 0.9299 - 170s/epoch - 363ms/step
Epoch 3/10
469/469 - 174s - loss: 0.0907 - accuracy: 0.9649 - val_loss: 0.2396 - val_accuracy: 0.9311 - 174s/epoch - 371ms/step
Epoch 4/10
469/469 - 173s - loss: 0.0862 - accuracy: 0.9660 - val_loss: 0.2510 - val_accuracy: 0.9286 - 173s/epoch - 370ms/step
Epoch 5/10
469/469 - 170s - loss: 0.0843 - accuracy: 0.9672 - val_loss: 0.2541 - val_accuracy: 0.9295 - 170s/epoch - 362ms/step
Epoch 6/10
469/469 - 170s - loss: 0.0830 - accuracy: 0.9679 - val_loss: 0.2585 - val_accuracy: 0.9310 - 170s/epoch - 362ms/step
Epoch 7/10
469/469 - 170s - loss: 0.0797 - accuracy: 0.9686 - val_loss: 0.2621 - val_accuracy: 0.9326 - 170s/epoch - 362ms/step
Epoch 8/10
469/469 - 169s - loss: 0.0775 - accuracy: 0.9697 - val_loss: 0.2653 - val_accuracy: 0.9305 - 

In [ ]:
best_cnn= tf.keras.models.load_model('best_acc_cnn.h5')
res_best_acc_cnn=best_cnn.evaluate(x_test, y_test,verbose=0)
print("3. best model 정확률은", res_best_acc_cnn[1]*100)
print('  res_best_acc_cnn =', res_best_acc_cnn)

3. best model 정확률은 93.26000213623047
  res_best_acc_cnn = [0.2620628774166107, 0.9326000213623047]


In [ ]:
#최고성능 모델과 Early stopping을 결합해서 모델을 저장 및 불러오기

checkpoint_cb= tf.keras.callbacks.ModelCheckpoint(
    'best_acc_cnn.h5', save_best_only=True)
early_stopping_cb=tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

hist=cnn.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test),
             callbacks= [checkpoint_cb, early_stopping_cb],verbose=0)

best_ES_cnn= tf.keras.models.load_model("best_acc_cnn.h5")
res_best_ES_cnn=best_ES_cnn.evaluate(x_test, y_test, verbose=0)
print('5. best + early stop 정확률은', res_best_ES_cnn[1]*100)
print(' res_best_ES_cnn =', res_best_ES_cnn)


5. best + early stop 정확률은 93.11000108718872
 res_best_ES_cnn = [0.27693453431129456, 0.9311000108718872]


###전이학습 (14쪽 ~)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

(x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)   #1을 추가해서 28*28*1 해서 reshape많이해줌
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)   #원핫인코딩
y_test=tf.keras.utils.to_categorical(y_test,10)

#작은필터를 여러개 쓰는게 더 좋다
#필터의 개수를 계속 늘려가라
#->vggnet 핵심 개념

'''
cnn=Sequential()
cnn.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
cnn.add(Conv2D(64, (3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation='softmax'))
'''


In [ ]:
cnn_model= tf.keras.models.load_model('saved_cnn.h5')

In [ ]:
res_cnn=cnn_model.evaluate(x_test, y_test, verbose=0)
print("1. cnn model 정확률은",res_cnn[1]*100)
print(' res_cnn=', res_cnn, '\n')

In [ ]:
for curr_layer in cnn_model.layers:
  print(curr_layer.trainable, ':==', curr_layer)
  curr_layer.trainable=False

no_layer= len(cnn_model.layers)
cnn_model.layers[no_layer-2].trainable=True
cnn_model.layers[no_layer-1].trainable=True

print("\nLayers.trainable = ", end=' ')
for curr_layer in cnn_model.layers:
  print(curr_layer.trainable, end=' ')


In [ ]:
cnn_model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(0.002),
                  metrics=["accuracy"])

In [ ]:

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'transfer_best_cnn.h5', monitor='val_accuracy', save_best_only=True)

hist=cnn_model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test),
             callbacks=[checkpoint_callback], verbose=2)

In [ ]:
res_transfer_cnn=cnn_model.evaluate(x_test, y_test, verbose=0)
print("2. transfered learning model 정확률은", res_transfer_cnn[1]*100)
print(' res_cnn= ', res_transfer_cnn, '\n')

In [ ]:
cnn_model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(),
                  metrics=["accuracy"])

In [ ]:

hist=cnn_model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test),
             callbacks=[checkpoint_callback], verbose=2)

cnn.save('saved_cnn.h5')

In [ ]:
cnn.summary()

In [ ]:
cnn_model=tf.keras.models.load_model('saved_cnn.h5')
cnn_model.summary()

In [ ]:
cnn_model.pop()
cnn_model.pop()
cnn_model.pop()
cnn_model.summary()

In [ ]:
#출력이 5개만을 분류하도록 변경함.
cnn_model.add(Dense(645))
cnn_model.add(Dense(5, activation='softmax'))
cnn_model.summary()

###Example 3-12:전이학습:새의 품종 분류 (27쪽~)

In [ ]:
코드 주고 출력 맞추기, 코드 메꾸기
레이어 중에 학습 안되는 레이어 뭐냐
수업 코드들 중요
활성함수 보고 회귀 or 분류 등
녹음듣

범위: 판다스~끝까지
